Define Evaluation Metrics

Since  predicting a continuous variable (block_median_fee_rate), regression metrics like:

Mean Absolute Error (MAE)

Root Mean Squared Error (RMSE)

R² Score

Neural net works- LSTM

In [1]:
!pip install scikeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 54.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip uninstall scikit-learn
!pip install scikit-learn==1.2.2
!pip install scikeras
!pip install tensorflow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 25.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
scikeras

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

In [4]:
df = pd.read_csv('/kaggle/input/bitcoin-history1-clean/bitcoin_history1_clean.csv')
df = df.set_index('block_time')

# List of fee rate columns that need to be converted from BTC/vB to sat/vB
fee_rate_columns = ['max_fee_rate', 'avg_fee_rate', 'median_fee_rate', 'fee_rate_10th', 'fee_rate_90th', 'fee_rate_std', 'block_median_fee_rate']


for col in fee_rate_columns:
    df[col] = df[col] * 10**8


# Define features (X) and target (y)
X = df.drop(columns=['block_median_fee_rate'])
y = df['block_median_fee_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  2


In [6]:
!nvidia-smi


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Sat Sep 28 03:08:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8             10W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
# 3. LSTM Model without fine tuning
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Reshape data for LSTM
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

lstm_model = create_lstm_model((X_train_lstm.shape[1], 1))
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=2)
lstm_y_pred = lstm_model.predict(X_test_lstm)
results={}
results['LSTM (without fine tuning)'] = {
    'MAE': mean_absolute_error(y_test, lstm_y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test, lstm_y_pred)),
    'R²': r2_score(y_test, lstm_y_pred)
}

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
668/668 - 6s - 10ms/step - loss: 5917.8306
Epoch 2/10
668/668 - 3s - 5ms/step - loss: 4805.9033
Epoch 3/10
668/668 - 3s - 5ms/step - loss: 4627.1963
Epoch 4/10
668/668 - 3s - 5ms/step - loss: 4613.1577
Epoch 5/10
668/668 - 3s - 5ms/step - loss: 4612.9082
Epoch 6/10
668/668 - 5s - 8ms/step - loss: 3656.9553
Epoch 7/10
668/668 - 3s - 5ms/step - loss: 3318.8281
Epoch 8/10
668/668 - 3s - 5ms/step - loss: 3272.1143
Epoch 9/10
668/668 - 3s - 5ms/step - loss: 3253.9314
Epoch 10/10
668/668 - 3s - 5ms/step - loss: 3242.5674
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [8]:
results

{'LSTM (without fine tuning)': {'MAE': 34.58225648181667,
  'RMSE': 55.132894410946584,
  'R²': 0.30463170264614803}}

1. First Approach: Independent Samples with Multiple Features
In this approach, assume that each sample (e.g., each block in your case) is independent, and just reshape the data for LSTM input without considering any sequence.

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN-LSTM
X_train_cnn_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

def create_cnn_lstm_model(lstm_units=50, conv_filters=64, conv_kernel_size=3, pool_size=2,
                          dropout_rate=0.2, learning_rate=0.001, optimizer='adam'):
    model = Sequential()
    model.add(Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation='relu',
                     input_shape=(X_train_cnn_lstm.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units//2))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

# Create the KerasRegressor
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
cnn_lstm_regressor = KerasRegressor(model=create_cnn_lstm_model, verbose=0, callbacks=[early_stopping])

# Define the hyperparameter grid
param_grid = {
    'model__lstm_units': [32, 64],
    'model__conv_filters': [32, 64],
    'model__conv_kernel_size': [3],
    'model__pool_size': [2],
    'model__dropout_rate': [0.1, 0.2],
    'model__learning_rate': [0.001],
    'model__optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [50]
}


# Initialize and run RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=cnn_lstm_regressor, param_distributions=param_grid,
                                   n_iter=30, cv=3, verbose=2, random_state=42, n_jobs=1,
                                   scoring='neg_mean_squared_error')

random_search_result = random_search.fit(X_train_cnn_lstm, y_train)

# Print the best parameters and score
print("Best Parameters: ", random_search_result.best_params_)
print("Best MSE: ", -random_search_result.best_score_)

# Evaluate on the test set
best_cnn_lstm_model = random_search_result.best_estimator_
cnn_lstm_y_pred = best_cnn_lstm_model.predict(X_test_cnn_lstm)



In [ ]:
# Calculate performance metrics
mae = mean_absolute_error(y_test, cnn_lstm_y_pred)
rmse = np.sqrt(mean_squared_error(y_test, cnn_lstm_y_pred))
r2 = r2_score(y_test, cnn_lstm_y_pred)

results['LSTM (tuned)'] = {
    'MAE': mae,
    'RMSE': rmse ,
    'R²': r2
}

# save the best model
best_cnn_lstm_model.model_.save('best_cnn_lstm_model.h5')

In [ ]:
results

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN-LSTM
X_train_cnn_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

def create_cnn_lstm_model(lstm_units=50, conv_filters=64, conv_kernel_size=3, pool_size=2,
                          dropout_rate=0.2, learning_rate=0.001, optimizer='adam'):
    model = Sequential()
    model.add(Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation='relu',
                     input_shape=(X_train_cnn_lstm.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units//2))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

# Create the KerasRegressor
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
cnn_lstm_regressor = KerasRegressor(model=create_cnn_lstm_model, verbose=0, callbacks=[early_stopping])

# Define the hyperparameter grid
param_grid = {
    'model__lstm_units': [64],  # Fixed for now
    'model__conv_filters': [64],  # Fixed for now
    'model__conv_kernel_size': [3],
    'model__pool_size': [2, 3],
    'model__dropout_rate': [0.1, 0.2, 0.3],
    'model__learning_rate': [0.001, 0.0005, 0.0001],
    'model__optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64],
    'epochs': [50,100]
}


# Initialize and run RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=cnn_lstm_regressor, param_distributions=param_grid,
                                   n_iter=30, cv=3, verbose=2, random_state=42, n_jobs=1,
                                   scoring='neg_mean_squared_error')

random_search_result = random_search.fit(X_train_cnn_lstm, y_train)

# Print the best parameters and score
print("Best Parameters: ", random_search_result.best_params_)
print("Best MSE: ", -random_search_result.best_score_)

# Evaluate on the test set
best_cnn_lstm_model = random_search_result.best_estimator_
cnn_lstm_y_pred = best_cnn_lstm_model.predict(X_test_cnn_lstm)



In [ ]:
# Calculate performance metrics
mae = mean_absolute_error(y_test, cnn_lstm_y_pred)
rmse = np.sqrt(mean_squared_error(y_test, cnn_lstm_y_pred))
r2 = r2_score(y_test, cnn_lstm_y_pred)

results['LSTM (tuned1)'] = {
    'MAE': mae,
    'RMSE': rmse ,
    'R²': r2
}



In [ ]:
results

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN-LSTM
X_train_cnn_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

def create_cnn_lstm_model(lstm_units=50, conv_filters=64, conv_kernel_size=3, pool_size=2,
                          dropout_rate=0.2, learning_rate=0.001, optimizer='adam'):
    model = Sequential()
    model.add(Conv1D(filters=conv_filters, kernel_size=conv_kernel_size, activation='relu',
                     input_shape=(X_train_cnn_lstm.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units//2))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) 

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

# Create the KerasRegressor
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
cnn_lstm_regressor = KerasRegressor(model=create_cnn_lstm_model, verbose=0, callbacks=[early_stopping])

# Define the hyperparameter grid
param_grid = {
    'model__lstm_units': [64, 128, 256],  
    'model__conv_filters': [64, 128, 256], 
    'model__conv_kernel_size': [3],
    'model__pool_size': [3,4],
    'model__dropout_rate': [0.2],
    'model__learning_rate': [0.001,0.005,0.01],
    'model__optimizer': [ 'rmsprop'],
    'batch_size': [16,32],
    'epochs': [50,100]
}


# Initialize and run RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=cnn_lstm_regressor, param_distributions=param_grid,
                                   n_iter=30, cv=3, verbose=2, random_state=42, n_jobs=1,
                                   scoring='neg_mean_squared_error')

random_search_result = random_search.fit(X_train_cnn_lstm, y_train)

# Print the best parameters and score
print("Best Parameters: ", random_search_result.best_params_)
print("Best MSE: ", -random_search_result.best_score_)

# Evaluate on the test set
best_cnn_lstm_model = random_search_result.best_estimator_
cnn_lstm_y_pred = best_cnn_lstm_model.predict(X_test_cnn_lstm)



In [ ]:
# Define the hyperparameter grid best
param_grid = {
    'model__lstm_units': [128], 
    'model__conv_filters': [128],
    'model__conv_kernel_size': [3],
    'model__pool_size': [3],
    'model__dropout_rate': [0.2],
    'model__learning_rate': [0.001],
    'model__optimizer': [ 'rmsprop'], 
    'batch_size': [32],
    'epochs': [100]
}

LSTM (tuned2) fine tuning

In [41]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense,LayerNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN-LSTM
X_train_cnn_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Define the CNN-LSTM model using the best hyperparameters
def create_cnn_lstm_model():
    model = Sequential()
    
    # First Conv1D layer
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train_cnn_lstm.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    
#     # Second Conv1D layer
#     model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=3))
    
    # First LSTM layer
    model.add(LSTM(units=128, return_sequences=True))
    model.add(LayerNormalization())
    model.add(Dropout(0.2))
    
    # Second LSTM layer
    model.add(LSTM(units=64, return_sequences=False))
    model.add(LayerNormalization())
    model.add(Dropout(0.2))
    
    # Output layer
    model.add(Dense(1))
    
    # Optimizer
    opt = RMSprop(learning_rate=0.001)
    
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    return model

# Instantiate the model
cnn_lstm_model = create_cnn_lstm_model()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
cnn_lstm_model.fit(X_train_cnn_lstm, y_train, 
                   validation_split=0.2, 
                   epochs=100, 
                   batch_size=32, 
                   callbacks=[early_stopping], 
                   verbose=2)

# Evaluate on the test set
cnn_lstm_y_pred = cnn_lstm_model.predict(X_test_cnn_lstm)




Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 - 5s - 10ms/step - loss: 4867.4746 - val_loss: 3026.1741
Epoch 2/100
534/534 - 3s - 5ms/step - loss: 2363.5022 - val_loss: 1863.6954
Epoch 3/100
534/534 - 3s - 5ms/step - loss: 1668.5961 - val_loss: 1389.2795
Epoch 4/100
534/534 - 3s - 5ms/step - loss: 1351.8878 - val_loss: 1201.0652
Epoch 5/100
534/534 - 3s - 5ms/step - loss: 1159.7240 - val_loss: 1058.0433
Epoch 6/100
534/534 - 3s - 6ms/step - loss: 1039.9971 - val_loss: 963.2625
Epoch 7/100
534/534 - 3s - 5ms/step - loss: 964.7334 - val_loss: 905.6763
Epoch 8/100
534/534 - 3s - 5ms/step - loss: 921.7800 - val_loss: 857.5228
Epoch 9/100
534/534 - 3s - 5ms/step - loss: 862.7465 - val_loss: 819.1367
Epoch 10/100
534/534 - 3s - 5ms/step - loss: 824.4869 - val_loss: 782.0953
Epoch 11/100
534/534 - 3s - 5ms/step - loss: 766.6662 - val_loss: 836.4711
Epoch 12/100
534/534 - 3s - 5ms/step - loss: 747.6499 - val_loss: 726.2568
Epoch 13/100
534/534 - 3s - 6ms/step - loss: 704.7455 - val_loss: 716.9762
Epoch 14/100
534/534 - 3s - 5ms/st

In [42]:
# Calculate performance metrics
mae = mean_absolute_error(y_test, cnn_lstm_y_pred)
rmse = np.sqrt(mean_squared_error(y_test, cnn_lstm_y_pred))
r2 = r2_score(y_test, cnn_lstm_y_pred)

results['LSTM (tuned2)'] = {
    'MAE': mae,
    'RMSE': rmse ,
    'R²': r2
}


In [43]:
results

{'LSTM (without fine tuning)': {'MAE': 34.58225648181667,
  'RMSE': 55.132894410946584,
  'R²': 0.30463170264614803},
 'LSTM (tuned3)': {'MAE': 9.33564540674665,
  'RMSE': 18.266595049879566,
  'R²': 0.9236676728626771},
 'LSTM (tuned2)': {'MAE': 10.182954115933548,
  'RMSE': 19.686484439226202,
  'R²': 0.9113396101606775}}

In [25]:
# Best model
# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN-LSTM
X_train_cnn_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Define the CNN-LSTM model using the best hyperparameters
def create_cnn_lstm_model():
    model = Sequential()
    
    # First Conv1D layer
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu',
                     input_shape=(X_train_cnn_lstm.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    
    # First LSTM layer
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.2))
    
    # Second LSTM layer
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.2))
    
    
    # Output layer
    model.add(Dense(1))
    
    # Optimizer (using the best learning rate and RMSProp optimizer)
    opt = RMSprop(learning_rate=0.001)
    
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    return model

# Instantiate the model
cnn_lstm_model = create_cnn_lstm_model()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
cnn_lstm_model.fit(X_train_cnn_lstm, y_train, 
                   validation_split=0.2, 
                   epochs=100, 
                   batch_size=32, 
                   callbacks=[early_stopping], 
                   verbose=2)

# Evaluate on the test set
cnn_lstm_y_pred = cnn_lstm_model.predict(X_test_cnn_lstm)




Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 - 4s - 8ms/step - loss: 5566.6694 - val_loss: 4458.9009
Epoch 2/100
534/534 - 3s - 5ms/step - loss: 3863.8911 - val_loss: 3218.9304
Epoch 3/100
534/534 - 2s - 5ms/step - loss: 2892.6917 - val_loss: 2507.4495
Epoch 4/100
534/534 - 3s - 5ms/step - loss: 2355.0793 - val_loss: 2081.6624
Epoch 5/100
534/534 - 2s - 5ms/step - loss: 1983.8469 - val_loss: 1814.8898
Epoch 6/100
534/534 - 3s - 5ms/step - loss: 1726.3030 - val_loss: 1545.6713
Epoch 7/100
534/534 - 3s - 5ms/step - loss: 1473.6467 - val_loss: 1352.4801
Epoch 8/100
534/534 - 3s - 5ms/step - loss: 1308.9408 - val_loss: 1255.3600
Epoch 9/100
534/534 - 2s - 5ms/step - loss: 1148.1788 - val_loss: 1133.8040
Epoch 10/100
534/534 - 3s - 5ms/step - loss: 1041.5220 - val_loss: 1024.7556
Epoch 11/100
534/534 - 2s - 5ms/step - loss: 966.8279 - val_loss: 933.0691
Epoch 12/100
534/534 - 2s - 5ms/step - loss: 902.5605 - val_loss: 947.4424
Epoch 13/100
534/534 - 2s - 5ms/step - loss: 841.0476 - val_loss: 776.9385
Epoch 14/100
534/534 - 3s 

In [26]:
# Calculate performance metrics
mae = mean_absolute_error(y_test, cnn_lstm_y_pred)
rmse = np.sqrt(mean_squared_error(y_test, cnn_lstm_y_pred))
r2 = r2_score(y_test, cnn_lstm_y_pred)

results['LSTM (tuned3)'] = {
    'MAE': mae,
    'RMSE': rmse ,
    'R²': r2
}


In [27]:
results

{'LSTM (without fine tuning)': {'MAE': 34.58225648181667,
  'RMSE': 55.132894410946584,
  'R²': 0.30463170264614803},
 'LSTM (tuned3)': {'MAE': 9.33564540674665,
  'RMSE': 18.266595049879566,
  'R²': 0.9236676728626771},
 'LSTM (tuned2)': {'MAE': 10.2362190709499,
  'RMSE': 18.56297168570665,
  'R²': 0.921170584707235}}